## Introducción a Pyspark (I): Sintáxis básica


El siguiente notebook contiene un pequeño tutorial para explicar los conceptos más básicos de sintaxis de Pyspark que nos permitan luego ver un ejemplo de construcción de modelo y que el alumno pueda profundizar por su cuenta.

Como es un notebook preparado para ser ejecutado en "Databricks", contiene código que es específico para esta plataforma. 

### Inciso DataBricks (I): Crear un cluster y asociar el notebook a dicho clúster

Antes de poder ejecutar cualquier código en Databricks es necesario tener un clúster al que poder asociar dicho código. Si ejectuas la celda que viene a continuación y no has asociado este notebook a un cluster te pedirá hacerlo. En clase vamos a ver cómo crear el cluster por "fuera" y luego asociarlo al notebook

In [0]:
import pyspark

Importado pyspark, lo primero como ya comentamos en las sesiones téoricas es crear ese "contexto" o "cursor" con el que poder interactuar con nuestro clúster:

In [0]:
from pyspark.sql import SparkSession

### Inciso DataBricks (II): Lectura de tablas y ficheros, empezando por Pandas

Vamos a leer ahora el primer fichero de datos que nos servirá como guía durante estas sesiones introductorias, pero para ello necesitamos saber dónde está (en la sesión anterior lo creamos y vimos el "path", pero seguramente no te acuerdas). Para saber en Databricks dónde están las tablas, ejecutamos la siguiente celda:

Comprobamos que tenemos un Unity Catalogue disponible.

In [0]:
%sql
SELECT CURRENT_METASTORE();

CURRENT_METASTORE()
aws:eu-west-1:cd71302f-5fd2-45a0-9baa-991f2a7adc26


In [0]:
# display(dbutils.fs.ls("dbfs:/FileStore/tables/"))
# no accesible en Databricks Free Edition por ser serverless

Las tablas están en el sistema de ficheros propio de DataBricks que no es accesible directamente con comandos python (si con pyspark). Como en este tutorial vamos a ir comparando pandas con pyspark, necesitamos mover esas tablas a un directorio "local" al nodo driver del cluster y ahí lo leeremos con pandas y luego ya lo volcaremos en un dataframe de spark para ir haciendo la comparativa.

NOTA: Al ser copias en el sistema de ficheros local de un nodo del clúster, cuando este se desconecte y termine, se perderán y tendrás que volver a copiarlas del dbfs

Las celdas siguiente no la vas a necesitar si programas en pyspark, ojo, es sólo para este tutorial pero ahí te queda por si quieres 

In [0]:
#%fs cp dbfs:/FileStore/tables/test1.csv file:/tmp/test1.csv



In [0]:
#%fs cp dbfs:/FileStore/tables/test2.csv file:/tmp/test2.csv


In [0]:
#%fs cp dbfs:/FileStore/tables/test3.csv file:/tmp/test3.csv

Y ahora ya sí, podemos leer nuestro fichero en pandas

In [0]:
import pandas as pd
df = pd.read_csv("./data/test1.csv")


In [0]:
df

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Harsha,21,1,15000
5,Shubham,23,2,18000


### Comenzando con Spark: Lectura de datos y echar un vistazo

Como ya se comentó en las sesiones teóricas, lo PRIMERO siempre es abrir un contexto o una sesión con Spark. Hoy en día siempre una sesión porque eso nos evita tener que abrir un contexto para cada módulo de Spark diferente.


La sesión adenmás nos da acceso a los métodos y funciones de DataFrame y DataSet, que son los elementos sobre los que trabajaremos de una forma "parecida" a como hemos trabajado los Dataframes de Pandas

In [0]:
spark = SparkSession.builder.appName("Practise").getOrCreate() # Fíjate en la sintaxis, no muy intuitiva. Se le da un nombre.

In [0]:
spark

Ahora vamos a leer el fichero a través de la interfaz de sparkm

In [0]:
#df_pyspark = spark.read.csv("/FileStore/tables/test1.csv")

In [0]:
%sql
SHOW VOLUMES

database,volume_name
default,albertovolume


In [0]:
!ls /Volumes/workspace/default/albertovolume

test1.csv  test2.csv  test3.csv


In [0]:
df_pyspark = spark.read.csv("/Volumes/workspace/default/albertovolume/test1.csv")

Recuerda que la lectura anterior es una transformación, no se ha ejecutado. 

Por eso ahora lo vamos a forar con el método show() (Que es como el head de pandas)

In [0]:
df_pyspark.show(2)

+-----+---+----------+------+
|  _c0|_c1|       _c2|   _c3|
+-----+---+----------+------+
| Name|age|Experience|Salary|
|Krish| 31|        10| 30000|
+-----+---+----------+------+
only showing top 2 rows


No lee los nombres de las columnas directamente como hace un read_csv porque no le hemos indicado nada al respecto. Ha considerado todas las filas como filas de datos y por eso la primera fila contine el nombre de las columnas. Veamos como forzar la lectura de la primera fila como nombre de columnas y otra forma de hacer la lectura del fichero.

In [0]:
df_pyspark = spark.read.csv("/Volumes/workspace/default/albertovolume/test1.csv", header= True)
df_pyspark_2 = spark.read.option("header","true").csv("/Volumes/workspace/default/albertovolume/test1.csv")

In [0]:
df_pyspark

DataFrame[Name: string, age: string, Experience: string, Salary: string]

In [0]:
display(df_pyspark.show())

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [0]:
df_pyspark_2.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [0]:
type(df_pyspark_2)

pyspark.sql.connect.dataframe.DataFrame

El comando análogo de info en pandas es printSchema. Ambos nos muestran las columnas, si pueden contener nulos y el tipo de cada columna

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        6 non-null      object
 1   age         6 non-null      int64 
 2   Experience  6 non-null      int64 
 3   Salary      6 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 324.0+ bytes


In [0]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



El atributo dtypes de un DataFrame de Pandas hace algo parecido también:


In [0]:
display(df.dtypes) # display hace que se vea un poco más estético pero no es necesario

Name          object
age            int64
Experience     int64
Salary         int64
dtype: object

Pero fijate en la diferencia... En spark todas las columnas son tipo string. A diferencia de pandas, spark si no se lo dices no hace inferencia de tipos. Tenemos que forzarlo como con las cabeceras

In [0]:
df_pyspark_3 = spark.read.csv("/Volumes/workspace/default/albertovolume/test3.csv", header = True, inferSchema= True) # Si no le dices nada a Databricks lee directamente de dbfs con los read de spark

In [0]:
df_pyspark_3.show()

+---------+------------+------+
|     Name| Departments|Salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [0]:
df_pyspark_3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- Salary: integer (nullable = true)



Como puedes ver ahora sí ha cogido los tipos de cada columna. Veamos que también se puede aplicar de la forma alternativa (y ya de paso que podemos leer también del sistema de ficheros locales del nodo driver)

In [0]:
df_pyspark_4 = spark.read.option("header","true").option("inferSchema","true").csv("/Volumes/workspace/default/albertovolume/test1.csv") # Para acceder en Databricks al sistema de ficheros local de un nodo hay que poner file: delante del path

In [0]:
df_pyspark_4.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



Los dataframes de Spark también tienen el atributo dtypes

In [0]:
df_pyspark_3.dtypes

[('Name', 'string'), ('Departments', 'string'), ('Salary', 'int')]

Para terminar la sesión, completemos el vistazo con el método describe (que existe tanto en Pandas con en Pyspark)

In [0]:
df.describe()

,age,Experience,Salary
count,6.000000,6.000000,6.000000
mean,26.333333,4.666667,21333.333333
std,4.179314,3.559026,5354.126135
min,21.000000,1.000000,15000.000000
25%,23.250000,2.250000,18500.000000
50%,26.500000,3.500000,20000.000000
75%,29.750000,7.000000,23750.000000
max,31.000000,10.000000,30000.000000


In [0]:
df_pyspark_4.describe().show() # Show fuerza a mostrar por pantalla y por tanto a "obtener" el dataframe, show es una acción (action)

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  NULL|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  NULL| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+



Aunque, se suele usar show para enseñar una muestra de datos de un dataframe, estos también tienen el método head (aunque su valor por defefcto es 1)

In [0]:
df.head()

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Harsha,21,1,15000


In [0]:
df.head(4)

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000


Y vamos bien servidos por ahora. En la siguiente sección/sesión avanzaremos en la selección, indexación y manipulación de columnas de un dataframe Spark. Recuerda que si dejas pasar más de una hora, el cluster se desconectará y para la siguiente sesión tendrás que crear otro nuevo y volver a ejectuar el código hasta aquí. 

***

### Intro Spark (II): Trabajando con columnas

#### Selección de Columnas

Empezamos viendo como se selecciona una columna de un dataframe. Primero con pandas y luego con spark

In [0]:
display(df["Name"])

0        Krish
1    Sudhanshu
2        Sunny
3         Paul
4       Harsha
5      Shubham
Name: Name, dtype: object

In [0]:
df_pyspark.select("Name")

DataFrame[Name: string]

Vuelve a ocurrirnos lo que nos pasaba con el display, select es una transformacion (fíjate además que en Pandas se origina una Serie y en spark se origina un DataFrame) y si no va seguida de un acción relacionada no se ejecuta.

En realidad todavía no ha hecho la operación, está esperando a que "sea necesario"

In [0]:
df_pyspark_3.select("Name").show()

+---------+
|     Name|
+---------+
|    Krish|
|    Krish|
|   Mahesh|
|    Krish|
|   Mahesh|
|Sudhanshu|
|Sudhanshu|
|Sudhanshu|
|    Sunny|
|    Sunny|
+---------+



Para seleccionar varias

In [0]:
df[["Name","Salary"]]

,Name,Salary
0,Krish,30000
1,Sudhanshu,25000
2,Sunny,20000
3,Paul,20000
4,Harsha,15000
5,Shubham,18000


In [0]:
df_pyspark_3.select(["Name","Salary"]).show()

+---------+------+
|     Name|Salary|
+---------+------+
|    Krish| 10000|
|    Krish|  5000|
|   Mahesh|  4000|
|    Krish|  4000|
|   Mahesh|  3000|
|Sudhanshu| 20000|
|Sudhanshu| 10000|
|Sudhanshu|  5000|
|    Sunny| 10000|
|    Sunny|  2000|
+---------+------+



#### Indexado por posición de columna y fila

Por indice de columna

In [0]:
df.columns[:2]

Index(['Name', 'age'], dtype='object')

In [0]:
df.columns[:2]

Index(['Name', 'age'], dtype='object')

In [0]:
df.iloc[:,:2]

,Name,age
0,Krish,31
1,Sudhanshu,30
2,Sunny,29
3,Paul,24
4,Harsha,21
5,Shubham,23


In [0]:
df_pyspark_3.select(df_pyspark_3.columns[:2]).show()

+---------+------------+
|     Name| Departments|
+---------+------------+
|    Krish|Data Science|
|    Krish|         IOT|
|   Mahesh|    Big Data|
|    Krish|    Big Data|
|   Mahesh|Data Science|
|Sudhanshu|Data Science|
|Sudhanshu|         IOT|
|Sudhanshu|    Big Data|
|    Sunny|Data Science|
|    Sunny|    Big Data|
+---------+------------+



La cosa se "complica" cuando queremos trabajar por ejemplo con índices posicionales (porque Pandas tiene siempre un índice posicional implicito), en el caso de Spark si lo necesitamos tenemos que crearlo.

In [0]:
df.iloc[2:4,:2]

,Name,age
2,Sunny,29
3,Paul,24


In [0]:
df_pyspark_3.select(df_pyspark_3.columns[:2]).show(3)

+------+------------+
|  Name| Departments|
+------+------------+
| Krish|Data Science|
| Krish|         IOT|
|Mahesh|    Big Data|
+------+------------+
only showing top 3 rows


Como ves no es lo mismo, para poder hacer algo similar tenemos que crear una columna que contenga un índice hecho "a mano". Para ello veamos primero como añadir una columna a un dataframe Spark.

#### Creando una nueva columna

En Pandas:

In [0]:
df["new_column"] = df["Experience"] + 2
df

,Name,age,Experience,Salary,new_column
0,Krish,31,10,30000,12
1,Sudhanshu,30,8,25000,10
2,Sunny,29,4,20000,6
3,Paul,24,3,20000,5
4,Harsha,21,1,15000,3
5,Shubham,23,2,18000,4


En spark hay que usar el metodo withColumn

In [0]:
df_pyspark_4 = df_pyspark_4.withColumn("new_column", df_pyspark_4["Experience"] + 2)

Recuerda que todavía no la ha creado

In [0]:
df_pyspark_4.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- new_column: integer (nullable = true)



In [0]:
df_pyspark_4.show()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|new_column|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|        12|
|Sudhanshu| 30|         8| 25000|        10|
|    Sunny| 29|         4| 20000|         6|
|     Paul| 24|         3| 20000|         5|
|   Harsha| 21|         1| 15000|         3|
|  Shubham| 23|         2| 18000|         4|
+---------+---+----------+------+----------+



El show tarda un poco más de lo "esperable", precisamente porque es cuando está creando el nuevo dataframe con la columna añadida

Cremos ahora un índice y simulemos un iloc (usando filter, un método que veremos un poco más en la siguiente sección/sesión): 

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
df_pyspark_4 = df_pyspark_4.withColumn("index", monotonically_increasing_id() )

In [0]:
df_pyspark_4.show()

+---------+---+----------+------+----------+-----+
|     Name|age|Experience|Salary|new_column|index|
+---------+---+----------+------+----------+-----+
|    Krish| 31|        10| 30000|        12|    0|
|Sudhanshu| 30|         8| 25000|        10|    1|
|    Sunny| 29|         4| 20000|         6|    2|
|     Paul| 24|         3| 20000|         5|    3|
|   Harsha| 21|         1| 15000|         3|    4|
|  Shubham| 23|         2| 18000|         4|    5|
+---------+---+----------+------+----------+-----+



In [0]:
df_pyspark_4.select(["Name","age","index"]).filter(df_pyspark_4["index"].between(2,3)).show()

+-----+---+-----+
| Name|age|index|
+-----+---+-----+
|Sunny| 29|    2|
| Paul| 24|    3|
+-----+---+-----+



Pero no teníamos que usar "select", ahí pone df_pyspark_4["index"]... Sí pero eso es porque es un objeto columna que no tiene métodos para visualizarse (solo los dataframe lo tienen), pero que sí puede usarse para generar condiciones como veremos cuando tratemos el metodo filter y las formas de seleccionar columnas en un dataframe de spark.

In [0]:
type(df_pyspark_4["index"])

pyspark.sql.connect.column.Column

Volveremos sobre ello, ahora veamos como eliminar y renombrar columnas:

### Eliminando columnas

In [0]:
df.drop("new_column", axis = 1)

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Harsha,21,1,15000
5,Shubham,23,2,18000


In [0]:
df_pyspark_4.drop("Experience")

DataFrame[Name: string, age: int, Salary: int, new_column: int, index: bigint]

En ambos casos no es una operación que "mute" el dataframe original (en Pandas es necesario el argumento inplace y en Spark no es posible hacerlo por filosofía de programación interna)

In [0]:
df_pyspark_4.show()

+---------+---+----------+------+----------+-----+
|     Name|age|Experience|Salary|new_column|index|
+---------+---+----------+------+----------+-----+
|    Krish| 31|        10| 30000|        12|    0|
|Sudhanshu| 30|         8| 25000|        10|    1|
|    Sunny| 29|         4| 20000|         6|    2|
|     Paul| 24|         3| 20000|         5|    3|
|   Harsha| 21|         1| 15000|         3|    4|
|  Shubham| 23|         2| 18000|         4|    5|
+---------+---+----------+------+----------+-----+



Con el renombrado ocurre igual, es necesario o usar el argumento inplace(Pandas) o asignarle el resultado de la operación a una nueva variable

In [0]:
df.rename(columns={"Name": "Nombre"}, inplace = True)

In [0]:
df

,Nombre,age,Experience,Salary,new_column
0,Krish,31,10,30000,12
1,Sudhanshu,30,8,25000,10
2,Sunny,29,4,20000,6
3,Paul,24,3,20000,5
4,Harsha,21,1,15000,3
5,Shubham,23,2,18000,4


In [0]:
df_pyspark_4.withColumnRenamed("Name","Nombre").show()

+---------+---+----------+------+----------+-----+
|   Nombre|age|Experience|Salary|new_column|index|
+---------+---+----------+------+----------+-----+
|    Krish| 31|        10| 30000|        12|    0|
|Sudhanshu| 30|         8| 25000|        10|    1|
|    Sunny| 29|         4| 20000|         6|    2|
|     Paul| 24|         3| 20000|         5|    3|
|   Harsha| 21|         1| 15000|         3|    4|
|  Shubham| 23|         2| 18000|         4|    5|
+---------+---+----------+------+----------+-----+



In [0]:
df_pyspark_4.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- new_column: integer (nullable = true)
 |-- index: long (nullable = false)



In [0]:
df_renombrado = df_pyspark_4.withColumnRenamed("Name","Nombre")

In [0]:
df_renombrado_2 = df_renombrado.withColumnRenamed("Salary","Nomina")

In [0]:
df_renombrado_2.show()

+---------+---+----------+------+----------+-----+
|   Nombre|age|Experience|Nomina|new_column|index|
+---------+---+----------+------+----------+-----+
|    Krish| 31|        10| 30000|        12|    0|
|Sudhanshu| 30|         8| 25000|        10|    1|
|    Sunny| 29|         4| 20000|         6|    2|
|     Paul| 24|         3| 20000|         5|    3|
|   Harsha| 21|         1| 15000|         3|    4|
|  Shubham| 23|         2| 18000|         4|    5|
+---------+---+----------+------+----------+-----+



Aquí terminamos la sección/sesión. En la siguiente veremos el tratamiento de valores faltantes y el filtrado y selección por valores. 

***

### Intro Spark (III): Missing Values y Filtrado por valores

#### Missing Values

Leemos primero una tabla con valores faltantes

In [0]:
df_pyspark_5 = pd.read_csv("./data/test2.csv")
df_pyspark_5

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0
6,Mahesh,NaN,NaN,40000.0
7,NaN,34.0,10.0,38000.0
8,NaN,36.0,NaN,NaN


In [0]:
df_pyspark_5 = spark.read.csv("/Volumes/workspace/default/albertovolume/test2.csv", header = True, inferSchema= True)

In [0]:
df_pyspark_5.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



Y aquí las opciones son las de siempre:
- Eliminar las filas con valores nulos
- Eliminar las columnas con valores nulos
- Imputar valores "representativos" (moda en categóricas, media en numéricas tipicamente)

1. Eliminando las filas con valores faltantes:

In [0]:
df_sin_missing = df_pyspark_5.na.drop()

In [0]:
df_sin_missing.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



2. Quedándonos con las columnas que no tienen missings o faltantes (imagina que "age" y "salary" no tienen valores faltantes)

In [0]:
df_sin_col_missing = df_pyspark_5.select(["age","Salary"])

In [0]:
df_sin_col_missing.show()

+----+------+
| age|Salary|
+----+------+
|  31| 30000|
|  30| 25000|
|  29| 20000|
|  24| 20000|
|  21| 15000|
|  23| 18000|
|NULL| 40000|
|  34| 38000|
|  36|  NULL|
+----+------+



Y antes de pasar a la imputación, veamos algunos de los parámetros de la función drop (parecidos a Pandas):


* how = "any" elimina la fila si uno de los valores es nulo (comportamiento por defecto)
* how = "all" elimina la fila si todos los valores son nulos  
* Otro parámetro es "tresh". Nos dice el número minimo de valores no nulos que teine que tener una fila antes de eliminarla. Por ejemplo tresh = 2, mantiene las columnas con al menos 2 valores NO NULOS

In [0]:
df_pyspark_5.na.drop(how = "any").show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [0]:
df_pyspark_5.na.drop(how = "all").show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
|     NULL|  36|      NULL|  NULL|
+---------+----+----------+------+



In [0]:
df_pyspark_5.na.drop(how = "any", thresh= 2).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     NULL|  34|        10| 38000|
+---------+----+----------+------+



Veamos how en pandas a modo de recordatorio:

In [0]:
df_2 = pd.read_csv("./data/test2.csv")

In [0]:
df_2

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0
6,Mahesh,NaN,NaN,40000.0
7,NaN,34.0,10.0,38000.0
8,NaN,36.0,NaN,NaN


In [0]:
df_2.dropna(how = "any")

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0


In [0]:
df_2.dropna(how = "all")

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0
6,Mahesh,NaN,NaN,40000.0
7,NaN,34.0,10.0,38000.0
8,NaN,36.0,NaN,NaN


Eliminando valores por columna, cuando sólo queremos eliminar aquellas filas en las que determinadas columnas tienen nulos

In [0]:
df_pyspark_5.na.drop(subset=("Experience")).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     NULL| 34|        10| 38000|
+---------+---+----------+------+



In [0]:
df_2.dropna(subset= "Experience")

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0
7,NaN,34.0,10.0,38000.0


Y para terminar el tratamiento de nulos, veamos la imputación de missing values:

In [0]:
df_pyspark_5.na.fill(20).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|   Mahesh| 20|        20| 40000|
|     NULL| 34|        10| 38000|
|     NULL| 36|        20|    20|
+---------+---+----------+------+



In [0]:
df_pyspark_5.na.fill("Pepe").show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|      NULL| 40000|
|     Pepe|  34|        10| 38000|
|     Pepe|  36|      NULL|  NULL|
+---------+----+----------+------+



Fíjate que sólo imputa en aquellas columnas cuyo tipo es compatible con el valor de relleno. En cambio en Pandas se fuerza la imputación:

In [0]:
df_2.fillna(20)

,Name,age,Experience,Salary
0,Krish,31.0,10.0,30000.0
1,Sudhanshu,30.0,8.0,25000.0
2,Sunny,29.0,4.0,20000.0
3,Paul,24.0,3.0,20000.0
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0
6,Mahesh,20.0,20.0,40000.0
7,20,34.0,10.0,38000.0
8,20,36.0,20.0,20.0


In [0]:
df_2.Experience.fillna(20)

0    10.0
1     8.0
2     4.0
3     3.0
4     1.0
5     2.0
6    20.0
7    10.0
8    20.0
Name: Experience, dtype: float64

Si lo que queremos es imputar valores según la columna:

In [0]:
df_pyspark_5.na.fill({"Name": "Ana", "Experience": 3}).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|NULL|         3| 40000|
|      Ana|  34|        10| 38000|
|      Ana|  36|         3|  NULL|
+---------+----+----------+------+



Pasemos ahora a filtrar (algo que vimos en la sesión anterior muy de pasada) y a aplicar condiciones (muy parecido a como las aplicamos con Pandas)

#### Filtrado y condiciones

Como vimos cuando intentamos hacer un iloc con dataframes de spark, el método para filtrar es filter y hay que pasarle una condición. De nuevo, filter es una transformación (su resultado es otro dataframe a partir del original) por lo que necesitamos aplicar una acción para forzar la ejecución. 

Veamos varias formas de proporcionar las condiciones:

* Como un string usando el nombre de las columnas

In [0]:
df_pyspark_5.filter("Salary <= 20000").show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



* Creando una mascara asignada a una variable, análogamente a como hemos hecho muchas veces con Pandas

In [0]:
condicion = df_2["Experience"] < 3
condicion_spark = df_pyspark_5["Experience"] < 3

In [0]:
df_2[condicion]

,Name,age,Experience,Salary
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0


De igual forma podemos combinar máscaras entre sí con los operadores & (and o y-lógico) y | (or-lógico)

In [0]:
condicion_spark_2 = (condicion_spark) & (df_pyspark_5["Salary"] > 10000)

In [0]:
condicion_2 = (condicion) & (df_2["Salary"] > 10000)
df_2[condicion_2]

,Name,age,Experience,Salary
4,Harsha,21.0,1.0,15000.0
5,Shubham,23.0,2.0,18000.0


Y también tenemos el operador negacion  ~, igual que en pandas

In [0]:
no_Harsha = ~(df_pyspark_5["Name"] == "Harsha")

Con esto damos por finalizada la sección. En la siguiente trataremos las agregaciones o agrupaciones (los group by) y la forma de operar sobre ellas

***

### Intro Spark (IV): Agrupaciones y agregaciones (groupby)

Para jugar con las agrupaciones en los dataframes de spark cargamos el tercer juego de datos

In [0]:
df_pyspark_6 = spark.read.csv("/Volumes/workspace/default/albertovolume/test3.csv", header = True, inferSchema= True)

In [0]:
df_pyspark_6.show()

+---------+------------+------+
|     Name| Departments|Salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



Al igual que con las agrupaciones en Pandas, es necesario aplicar una función de agregación para poder generar un dataframe final

In [0]:
df_grouped = df_pyspark_6.groupBy("Name")

In [0]:
df_grouped.sum().show()

+---------+-----------+
|     Name|sum(Salary)|
+---------+-----------+
|    Krish|      19000|
|Sudhanshu|      35000|
|    Sunny|      12000|
|   Mahesh|       7000|
+---------+-----------+



Y solo arrastra las columnas sobre las que tiene sentido la función de agregación

In [0]:
df_grouped.count().show()

+---------+-----+
|     Name|count|
+---------+-----+
|    Krish|    3|
|Sudhanshu|    3|
|    Sunny|    2|
|   Mahesh|    2|
+---------+-----+



Veamos la equivalencia en Pandas:

In [0]:
df_3 = pd.read_csv("./data/test3.csv")

In [0]:
df_3

,Name,Departments,Salary
0,Krish,Data Science,10000
1,Krish,IOT,5000
2,Mahesh,Big Data,4000
3,Krish,Big Data,4000
4,Mahesh,Data Science,3000
5,Sudhanshu,Data Science,20000
6,Sudhanshu,IOT,10000
7,Sudhanshu,Big Data,5000
8,Sunny,Data Science,10000
9,Sunny,Big Data,2000


In [0]:
pd_grouped = df_3.groupby("Name")

In [0]:
pd_grouped.sum(numeric_only= True)

,Salary
Name,
Krish,19000
Mahesh,7000
Sudhanshu,35000
Sunny,12000


Agrupando ahora por departamentos:

In [0]:
df_department = df_pyspark_6.groupBy("Departments")

In [0]:
df_department.count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|Data Science|    4|
|    Big Data|    4|
|         IOT|    2|
+------------+-----+



In [0]:
df_department.sum().show()

+------------+-----------+
| Departments|sum(Salary)|
+------------+-----------+
|Data Science|      43000|
|    Big Data|      15000|
|         IOT|      15000|
+------------+-----------+



In [0]:
df_department.mean().show()

+------------+-----------+
| Departments|avg(Salary)|
+------------+-----------+
|Data Science|    10750.0|
|    Big Data|     3750.0|
|         IOT|     7500.0|
+------------+-----------+



Y en pandas:

In [0]:
pd_departments = df_3.groupby("Departments")

In [0]:
pd_departments.count()

,Name,Salary
Departments,,
Big Data,4,4
Data Science,4,4
IOT,2,2


In [0]:
pd_departments.sum()

,Salary
Departments,
Big Data,15000
Data Science,43000
IOT,15000


In [0]:
pd_departments.mean(numeric_only= True)

,Salary
Departments,
Big Data,3750.0
Data Science,10750.0
IOT,7500.0


#### AGGREGATE

Al igual que en Pandas, se pueden aplicar diferentes funciones de agregación a una agrupación en función de la columna objetivo. Aunque primero veamos que ocurre cuando lo hacemos sin agrupar (que se aplica a todas las filas)

In [0]:
df_pyspark_6.agg({"Salary": "sum"}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+



Ahora ya sobre la agrupación

In [0]:
df_pyspark_6.groupBy("Departments").agg({"Salary": "sum"}).show()

+------------+-----------+
| Departments|sum(Salary)|
+------------+-----------+
|Data Science|      43000|
|    Big Data|      15000|
|         IOT|      15000|
+------------+-----------+



Recordando un poco de pandas:

In [0]:
pd_departments.agg(["sum","mean","count"])

Salary               
                sum     mean count
Departments                       
Big Data      15000   3750.0     4
Data Science  43000  10750.0     4
IOT           15000   7500.0     2

Un equivalente (mejorado):

In [0]:
df_pyspark_6.groupBy("Departments").agg({"Salary": "sum", "Salary": "mean", "Name": "count"}).show()

+------------+------------+-----------+
| Departments|mean(Salary)|count(Name)|
+------------+------------+-----------+
|Data Science|     10750.0|          4|
|    Big Data|      3750.0|          4|
|         IOT|      7500.0|          2|
+------------+------------+-----------+



Y ya casi estamos preparados para trabajar sobre nuestros dataframes como si fueran pandas, pero para terminar la introducción veremos en la siguiente sección como aplicar funciones a las columnas (y en particular las udf, user defined functions). Pero ya en la siguiente sección/sesión.

***

### Intro Spark (V): Funciones

Para empezar nos importamos unas cuantas funciones predefinidas que se pueden aplicar directamente a las columnas:

In [0]:
from pyspark.sql.functions import lower,col,ascii,mean

In [0]:
df_pyspark_6.show()

+---------+------------+------+
|     Name| Departments|Salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



Fijate en la sintaxis, que no es precisamente muy directa:

In [0]:
df_pyspark_6.select(lower(col("Name"))).show()

+-----------+
|lower(Name)|
+-----------+
|      krish|
|      krish|
|     mahesh|
|      krish|
|     mahesh|
|  sudhanshu|
|  sudhanshu|
|  sudhanshu|
|      sunny|
|      sunny|
+-----------+



In [0]:
df_pyspark_6.select(lower(df_pyspark_6["Name"])).show()

+-----------+
|lower(Name)|
+-----------+
|      krish|
|      krish|
|     mahesh|
|      krish|
|     mahesh|
|  sudhanshu|
|  sudhanshu|
|  sudhanshu|
|      sunny|
|      sunny|
+-----------+



Es necesario aplicar la función col primero (en el contexto de un método de un dataframe) y luego la función que queramos

Construimos ahora un dataframe con el código ascii del primer caracter del nombre (sí, no tiene ninguna utilidad aparente pero es que así practicamos)

In [0]:
df_test = df_pyspark_6.withColumn("New_Column", ascii(col("Name")))

In [0]:
df_test.show()

+---------+------------+------+----------+
|     Name| Departments|Salary|New_Column|
+---------+------------+------+----------+
|    Krish|Data Science| 10000|        75|
|    Krish|         IOT|  5000|        75|
|   Mahesh|    Big Data|  4000|        77|
|    Krish|    Big Data|  4000|        75|
|   Mahesh|Data Science|  3000|        77|
|Sudhanshu|Data Science| 20000|        83|
|Sudhanshu|         IOT| 10000|        83|
|Sudhanshu|    Big Data|  5000|        83|
|    Sunny|Data Science| 10000|        83|
|    Sunny|    Big Data|  2000|        83|
+---------+------------+------+----------+



Y ahora algo con más sentido obtener el salario medio:

In [0]:
valores = df_pyspark_6.select(mean(col("Salary")))

In [0]:
type(valores)

pyspark.sql.connect.dataframe.DataFrame

In [0]:
valores.printSchema()

root
 |-- avg(Salary): double (nullable = true)



In [0]:
valores.show()

+-----------+
|avg(Salary)|
+-----------+
|     7300.0|
+-----------+



#### User-Defined Function (udf): Funciones definidas por el usuario

Al igual que usando el método apply en pandas podíamos aplicar una función definida por nosotros mismos a una o varias columnas, una **UDF (User-Defined Function)** en PySpark es una función definida por el usuario que se puede aplicar a columnas de un DataFrame de Spark. Las UDFs permiten realizar operaciones personalizadas en los datos que no están disponibles de forma nativa en las funciones de PySpark.

Tienen alguna particularidad como es el registro de la función, por eso vamos a ver paso a paso como crear y usar una udf:
1. **Definir la Función en Python**: Escribes una función en Python que realice la operación deseada.
2. **Registrar la Función como UDF**: Usas `pyspark.sql.functions.udf` para convertir la función de Python en una UDF.
3. **Aplicar la UDF**: Aplicas la UDF a las columnas de un DataFrame de Spark.

Definimos una función que va a añadir un prefijo "bootcamp" a una columna string y a convertirla en 0 si es un número

In [0]:
def to_test(name):
    if type(name) == str:
        return f"bootcamp_{name}"
    else:
        return "0"

Ahora hay que registrarla

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Registrar la función como UDF
to_test_udf = udf(to_test, StringType())

Fijate que le decimos el tipo que retorna la función (Spark está basado en Scala y este lenguaje es mucho más riguroso con los tipos que python y exige este tipo de prevención respecto al tipo de salida, además de que casi todo es inmutable, con cierta flexibilidad)

Apliquemos la udf:

In [0]:
# Aplicar la UDF
df_with_upper = df_pyspark_6.withColumn("name_upper", to_test_udf(df_pyspark_6["Name"])) 

# Mostrar los resultados
df_with_upper.show()

+---------+------------+------+------------------+
|     Name| Departments|Salary|        name_upper|
+---------+------------+------+------------------+
|    Krish|Data Science| 10000|    bootcamp_Krish|
|    Krish|         IOT|  5000|    bootcamp_Krish|
|   Mahesh|    Big Data|  4000|   bootcamp_Mahesh|
|    Krish|    Big Data|  4000|    bootcamp_Krish|
|   Mahesh|Data Science|  3000|   bootcamp_Mahesh|
|Sudhanshu|Data Science| 20000|bootcamp_Sudhanshu|
|Sudhanshu|         IOT| 10000|bootcamp_Sudhanshu|
|Sudhanshu|    Big Data|  5000|bootcamp_Sudhanshu|
|    Sunny|Data Science| 10000|    bootcamp_Sunny|
|    Sunny|    Big Data|  2000|    bootcamp_Sunny|
+---------+------------+------+------------------+



In [0]:
df_with_upper = df_pyspark_6.withColumn("salary_upper", to_test_udf(df_pyspark_6["Salary"]))
df_with_upper.show() 

+---------+------------+------+------------+
|     Name| Departments|Salary|salary_upper|
+---------+------------+------+------------+
|    Krish|Data Science| 10000|           0|
|    Krish|         IOT|  5000|           0|
|   Mahesh|    Big Data|  4000|           0|
|    Krish|    Big Data|  4000|           0|
|   Mahesh|Data Science|  3000|           0|
|Sudhanshu|Data Science| 20000|           0|
|Sudhanshu|         IOT| 10000|           0|
|Sudhanshu|    Big Data|  5000|           0|
|    Sunny|Data Science| 10000|           0|
|    Sunny|    Big Data|  2000|           0|
+---------+------------+------+------------+



In [0]:
df_with_upper.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- salary_upper: string (nullable = true)



Y con esto terminamos la sesión introductoria a la sintaxis y al manejo más básico de dataframes Spark. En las siguientes sesiones veremos como construir un modelo supersencillo.

In [0]:
spark.stop() # Por ser un poquito ordenado, como nos ocurria con las bases de datos, una vez hemos terminado el trabajo es muy conveniente cerrar las sesiones